In [60]:
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=1a6bf03a4d1a27834753302f3c83b786591069dd1db05e4dc2f6793b5aaeddad
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [61]:
import openai
openai.api_key = ""

In [111]:
import time

In [103]:
classify = """
Classify wheather the lay definition is good and the general definition is appropriate.

Context : WEEK 2 Discontinue Lamotrigine WEEK 6 Start Aptiom 100 mg daily WEEK 8 Increase to Aptiom 200 mg daily. 
term : mg
general definition : this is short for milligram which is 1/1000 of a gram usually considered a small amount.
lay definition : A tiny amount of something, usually a drug.
answer : yes

context : Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitamin B complex with folic acid and vitamin c , take 1 tab po qd Allergies No Known Drug Allergies Recorded By : [ * * PERSON * * ] ; [ * * DATE * * ] 12:21:32 PM Vitals All Vital Signs [ Data Includes : Last 1 Instance ] * * Printed in Appendix # 1 below .
term : Virt - Vite
general definition : ['Virt', ' - ', 'The determination of the amount of Vitamin E present in a sample.']
lay definition : A mix of vitamins. It provides vitamin B-6, vitamin B-12 and folic acid to people who do not have enough of these for good health.
answer : no

Context : Biopsy of an enlarged inguinal lode on the left side showed metastatic adenocarcinoma.
term : inguinal
general definition : The external junctural region between the lower part of the abdomen and the thigh.
lay definition : To do with the area in between the fold of the groin.
answer : yes

context : Drug Allergies / ADR from Pharmacy : No Known Drug Allergies Allergies Comments : Note : Drug - induced dystonia from reglan or compazine may also be produced by the following : acetohenazine ( Tindal ) amoxapine ( Asendin ) chlorpromazine ( Thorazine ) fluphenazine ( Permitil , Prolixin ) haloperidol ( Haldol ) loxapine ( Loxitane , Daxolin ) mesoridazine ( Serentil ) metaclopramide ( Reglan ) molinndone ( [ * * PERSON * * ] ) perphanzine ( Trilafrom , Triavil ) piperacetazine ( Quide ) prochlorperzine ( Compazine , Combid ) promazine ( Sparine ) promethazine ( Phenergan ) thiethylperazine ( Torecan ) thioridazine ( Mellaril ) thiothixene ( Navane ) trifluoperazine ( Stelazine ) triflupromazine ( Vesprin ) trimeprazine ( Temaril ) .
term : prochlorperzine 
general definition : ['prochlorperzine', ' ']
lay definition : A drug used to prevent or reduce nausea and vomiting.
answer : no

context : PAST MEDICAL HISTORY : Anorexia nervosa , anxiety , bulimia nervosa , abdominal pain , aglossia , hyperlipidemia , hypokalemia , personal history of skin cancer .
term : aglossia
general definition : ['Absence of the tongue owing to a developmental abnormality. [HPO:probinson]']
lay definition : Being born without a tongue.
answer : yes

"""

Remove punctuations and do not defines and empty jterms or lay definitions.

In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df = pd.read_csv("case4.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df = df.drop('UMLS', axis=1)
df.head()


,Unnamed: 0,id,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
0,0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...","['Virt', ' - ', 'Vite']"
1,1,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,7.0,8.0,1,{DO NOT DEFINE},['1'],['1']
2,2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,"['MG', ' ']"
3,3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10.0,Oral,Taken by mouth.,['A substance intended for administration thro...,['Oral']
4,4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11.0,Tablet,A pill.,['A tablet intended for oral administration.'],['Tablet']


In [17]:
len(df)


348316

In [18]:
df = df.dropna()

In [19]:
len(df)

348316

In [21]:
df = df[df.text_to_annotate != ""]
len(df)

348316

In [25]:
df = df[df.text_to_annotate != ""]
df = df[df.ann_text != ""]
df = df[df.definition != "{DO NOT DEFINE}"]
df = df[df.definition != ""]

In [26]:
len(df)

312425

In [27]:
df.head()

,Unnamed: 0,id,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
0,0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...","['Virt', ' - ', 'Vite']"
2,2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,"['MG', ' ']"
3,3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10.0,Oral,Taken by mouth.,['A substance intended for administration thro...,['Oral']
4,4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11.0,Tablet,A pill.,['A tablet intended for oral administration.'],['Tablet']
5,5,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,11.0,14.0,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",['Vitamin B complex']


In [30]:
df = df[df.definition != " "]
df = df[df.definition != "  "]

In [31]:
len(df)

312425

In [32]:
import string

In [44]:
print(string.punctuation)

punc = [*string.punctuation]

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [51]:
#remove punctutations and empty cells

df = df[~df['definition'].isin(punc)]
df = df[~df['ann_text'].isin(punc)]

In [52]:
len(df)

309137

In [84]:
def data_gen(df,i):
  c = df.iloc[i]['text_to_annotate']
  t = df.iloc[i]['ann_text']
  g_d = df.iloc[i]['split_print']
  l_d = df.iloc[i]['definition'] 
  context = "context : " + c
  term = "term : " + t
  general_definition = "general definition : " + g_d
  lay_def = "lay definition : " + l_d
  # you can add answer to spend fewer tokens
  custom_prompt = context + "\n" + term + "\n" + general_definition + "\n"+lay_def + "\n"
  return custom_prompt


In [93]:
print(data_gen(df,1008))

context : Drug Allergies / ADR from Pharmacy : No Known Drug Allergies Allergies Comments : Note : Drug - induced dystonia from reglan or compazine may also be produced by the following : acetohenazine ( Tindal ) amoxapine ( Asendin ) chlorpromazine ( Thorazine ) fluphenazine ( Permitil , Prolixin ) haloperidol ( Haldol ) loxapine ( Loxitane , Daxolin ) mesoridazine ( Serentil ) metaclopramide ( Reglan ) molinndone ( [ * * PERSON * * ] ) perphanzine ( Trilafrom , Triavil ) piperacetazine ( Quide ) prochlorperzine ( Compazine , Combid ) promazine ( Sparine ) promethazine ( Phenergan ) thiethylperazine ( Torecan ) thioridazine ( Mellaril ) thiothixene ( Navane ) trifluoperazine ( Stelazine ) triflupromazine ( Vesprin ) trimeprazine ( Temaril ) .
term : prochlorperzine 
general definition : ['prochlorperzine', ' ']
lay definition : A drug used to prevent or reduce nausea and vomiting.



In [109]:
def output(prompt_t,custom_p,max_t = 10,temp = 0.1):
  prompt = prompt_t + custom_p
  completion = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=max_t, temperature=temp)

  # print("Generated Answer..")
  generated = completion.choices[0].text
  # print(generated)

  if "answer : " in generated:
    ind = generated.index("answer : ") 
    # print(generated[ind+9:])
  # return generated[ind+9:]

In [97]:
custom = data_gen(df,3)
output(classify,custom)

yes


'yes'

In [107]:
#1008,678,1006
custom = data_gen(df,678)
print(custom)
print(output(classify,custom))

context : HOSPITAL : [ * * LOC * * ] MEDICINE MID_LEVEL RAPID RESPONSE EVENT RECORD Date : [ * * DATE * * ] Time : 19:11 Hospital Day # : 3 Patient Name : Spahi , Aladin Date of Service : [ * * DATE * * ] Admission Date : [ * * DATE * * ] Location : M310 A Mid - Level Name : [ * * PERSON * * ] Pager # : [ * * MISC * * ] Account # : [ * * MISC * * ] Medical Record # : [ * * MISC * * ] DOB : [ * * DATE * * ] Age : [ * * MISC * * ] Sex : M RAPID RESPONSE TRIGGERS Marked nursing , housestaff , or family concern : True Marked nursing , housestaff , or family concern comment : Pt being unresponsive PHYSICAL EXAM VITALS Name [ * * DATE * * ] 17:46 Temp 36 .
term : PHYSICAL EXAM
general definition : ['Systematic and thorough inspection of the patient for physical signs of disease or abnormality.', ' EXAM']
lay definition : The act of a doctor inspecting the body of a patient for signs of disease.

yes
yes


In [110]:
df["reliable"] = ""

for i in range(len(df)):
  if i % 1000 == 0:
    print(i)
  custom = data_gen(df,i)
  df.at[i,"reliable"] = output(classify,custom)
  time.sleep(2)


0


RateLimitError: ignored

In [ ]:
df.to_csv("with_reliable")

In [47]:
df_m = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [",", 'Batmobile', 'Bullwhip'],
                   "born": [",", "new",
                            ","]})

In [48]:
df_m

,name,toy,born
0,Alfred,",",","
1,Batman,Batmobile,new
2,Catwoman,Bullwhip,","


In [49]:


df_m = df_m[~df_m['toy'].isin(punc)]

In [50]:
df_m

,name,toy,born
1,Batman,Batmobile,new
2,Catwoman,Bullwhip,","
